In [1]:
HOPSWORKS_PROJECT_NAME = "taxi_demand_jithis"
%reload_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import os
from pathlib import Path
from src.paths import PARENT_DIR

# Ensure PARENT_DIR is a Path object, not a string
parent_dir_path = Path(PARENT_DIR)

# Load environment variables from .env file
# Make sure the path is constructed correctly
dotenv_path = parent_dir_path / '.env'
load_dotenv(dotenv_path)

# Access API key with error handling
HOPSWORKS_API_KEY= os.getenv('HOPSWORKS_API_KEY')


In [3]:


from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])



File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
2023-11 file is not available
2023-12 file is not available


In [24]:


print(f'{len(rides)=:,}')



len(rides)=71,247,922


In [4]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)


100%|██████████| 265/265 [00:02<00:00, 121.99it/s]


In [5]:
import hopsworks

In [6]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)


Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.4) by running 'pip install hopsworks==3.4.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/325782


In [7]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


TypeError: FeatureStore.__init__() missing 3 required positional arguments: 'hdfs_store_path', 'featurestore_description', and 'inode_id'